**Learning Outcomes**

To understand: 
* What are functions?
* Function valence
* Defining and applying Functions
* Explicit and Implicit parameters
* Function Scope 
* How to create a projection
* Applying projections


# Introduction

Functions are key components for manipulating data in kdb+, they transform an input through a sequence of expressions which are evaluated and usually (but it's not required) the final output of these expressions is returned as the function output.  

# What are functions?


Intrinsically functions are described by the following: 
* Their input parameters (this can be none) - known as the function valence, kdb+/q functions can have up to 8 parameters
* The function logic - the code you write to make the function do what you want
* The function return (this can also be nothing) - what output is returned by the function

## Function Valence 
The function valence means the number of arguments, or input parameters a function needs to run. If you provide a lesser number of parameters than is required to a function you create a projection, if you provide more, you will get a `'rank`  error. 

To see what happens in these cases we can look at the in-built primitive `+` which takes two inputs: 

In [1]:
+[3;4]     //providing two parameters as expected

7


In [2]:
+[3;]      //providing just one parameter

+[3;]


In [2]:
+[3;4;1]   //providing three parameters

[0;31mrank[0m: [0;31mrank[0m

Binary operations (taking two parameters or inputs) in mathematics are called dyadic functions in q, and unary (taking one parameter or input) operations are called monadic functions. 


The minimum and maximum number of parameters to a function are zero and eight respectively. 

In [3]:
neg 1 2 3   //example of a monadic function 

-1 -2 -3


In [4]:
.Q.gc[]  //no input passed!- Niladic function - gets rid of all the data in the memory that is not assigned to anything

0


What would you expect if you called `neg` like `neg[1;2;3]`? How is that different from `neg[1 2 3]` or `neg 1 2 3`?

In [4]:
neg[1 2 3] //this would apply the neg operator on the vector
neg 1 2 3 //this would also apply the neg operator on the vector - different syntax
neg[1;2;3] //this would give a rank error as the semi colon delimits function parameters - passing 3 input parameters

-1 -2 -3


-1 -2 -3


[0;31mrank[0m: [0;31mrank[0m

##### Exercise

How many arguments does the operator [`mod`](https://code.kx.com/q/ref/mod/) have? 

In [ ]:
mod[10 15 20;10] // The correct answer is 2. 

In [5]:
//write your answer here 2

## Defining a function

The syntax of function definition is a matching pair of curly braces `{` and `}` - the smallest possible function we can define is actually the following: 

In [6]:
f:{}          //minimal function definition

In [7]:
type f        //lambda - user defined function

100h


In [8]:
f[]           //calling our function which takes no input

In [9]:
count f[]     

1


In [10]:
(::)~f[]      //nothing returned (::) is null return 

1b


In [ ]:
{}[]~f[]      //we don't have to assign the function to a variable to use it

### Function parameters 
We can specify parameters by using `[]` to enclose the list of our parameter values - the parameter names are then used throughout the rest of the function logic to reference the value passed for that input reference. 

In [11]:
f:{[parameter] parameter}                                 //function with one input
f[1]

1


In [12]:
divide:{[numerator;denominator] numerator%denominator}    //function with two inputs
divide[1]                                             //not enouogh param 

{[numerator;denominator] numerator%denominator}[1]


In [15]:
divide[1;2]~divide[1] 2  //function arguments are assigned in order of passing
divide[1;2]

1b


0.5


##### Exercise 

Define a function `range` that takes one parameter input `l` (a list) and doesn't do anything. 

What happens if you call this with one input e.g. `range[1]`? What about two inputs e.g. `range[1;2]`?

In [ ]:
range:{[l] }

In [ ]:
range 1  //the function doesn't return anything but it runs

In [ ]:
range[1;2]  //too many parameters - rank error 

In [19]:
//write your function here
range:{[l] }

In [20]:
range[1]

In [20]:
range[1;2]

[0;31mrank[0m: [0;31mrank[0m

### Function Logic 

The code we choose to include within our function is referred to as the function logic and is comprised of a series of q expressions. 

We can chain together our statements for code clarity and to allow us to work through multiple lines of expressions where needed:

In [21]:
convertToCelcius:{[farenheit] 
            offset:farenheit-32;   //defining the initial offset value
            offset*5%9           //multiplying by the appropriate ratio after offsetting
            }
convertToCelcius[70]  //translating the temperature from farenheit to celcius

21.11111


##### Exercise 

Define a function `range` that will calculate the range of numbers in a list. This function should have one parameter `l`, the list being passed. 

Calling `range` with an input -10 20 30 40 50  should return 60. 

In [ ]:
range:{[l] max[l]- min l}
range -10 20 30 40 50

In [24]:
//write your function here
range:{[l]
        min_l: min l;
        max_l: max l;
        max_l - min_l}
range[-10 20 30 40 50]

60


In [27]:
l: -10 20 30 40 50
min l
t: l - min l
max t

-10


60


### Returning from a Function 

We can see the functions `divide` and `convertToCelcius` above returned values - this is because the last q expression in each function definition wasn't suppressed by a semicolon.

In [28]:
convertToCelcius:{[farenheit] offset:farenheit-32;   
                            offset*5%9;             //adding a semicolon after the statement
                    }                               //no other expressions to evaluate
convertToCelcius[70]    

We can force an early return from a function by using `:` to indicate what we want to return. We can see this more clearly if we add a print statement to our function:

In [29]:
//Example 1 - output suppressed
convertToCelcius:{[farenheit] offset:farenheit-32;   
                            -1 "Finished offset, starting ratio conversion"; //printing to stdout to log
                            offset*5%9;             //no return
                    }                               
convertToCelcius[70]   //no return, but still prints

Finished offset, starting ratio conversion


In [34]:
//Example 2 - force return
convertToCelcius:{[farenheit] offset:farenheit-32;   
                            -1 "Finished offset, starting ratio conversion";
                            :offset*5%9;            //using : to force return   
                    }                               
convertToCelcius[70]   //returns the ouput and prints

Finished offset, starting ratio conversion


21.11111


In [35]:
//Example 3  - force return before print
convertToCelcius:{[farenheit] :offset:farenheit-32; //using : to force return   
                            -1 "Finished offset, starting ratio conversion";
                            offset*5%9;              
                    }                               
convertToCelcius[70]   //returns just offset value before print

38


In [37]:
//Example 4 - return without suppression 
convertToCelcius:{[farenheit] offset:farenheit-32; 
                            -1 "Finished offset, starting ratio conversion";
                            offset*5%9           //no force return, just returning the last expression     
                    }                               
convertToCelcius[70]   

Finished offset, starting ratio conversion


21.11111


If we look again at our `divide` function, we can use force return to return a default value if we pass a divisor of zero:

In [38]:
cappedDivide:{[numerator;denominator] 
                    if[denominator=0; :20f];  //if[1b; do these things], if[0b;they don't get done]
                    v:numerator%denominator} 
cappedDivide[100;20]
cappedDivide[100;0]

5f


20f


We could have avoided all of that hassle of force returning etc just by using the `min` function -  there are usually many ways to achieve the same thing!  

In [39]:
cappedDivide:{[numerator;denominator] 
                   min(20f;numerator%denominator)} 
cappedDivide[100;3]
cappedDivide[100;20]
cappedDivide[100;0]

20f


5f


20f


##### Exercise

Redefine the `range` function to return the result using the force return.  

In [ ]:
range:{[l] r:max[l]-min l; //assigning the result to a variable
    :r; //using the force return
 }
l:10?100  //defining a random list 
range[l]  //finding the range

In [40]:
//write your function here
range:{[l] r:max l - min l;
        :r;}
l: 10?100
range[l]

89


### Applying a function

When we apply a function, it causes the expressions in the function body to be evaluated in sequence, substituting the value of each argument for the corresponding input parameters. To pass parameters to a function we can use the functional syntax we've used before: 

In [41]:
{[a]a*a}[3]

square:{[a]a*a}
square[4]

9


16


One way in which user defined functions differ from native functions in kdb+/q is that we can't make use of the 'infix' notation that kdb+/q does - e.g. for a primitive with two inputs like `%` we can write this in a few ways:  

In [42]:
3 % 5       //infix notation 
%[3;5]      //functional notation
%[;5] 3     //eliding the first argument to pass the second 
%[3][5]     //projecting over the first argument and then calling with the second explicity 
%[3] 5      //projecting over the first argument and then applying implicity to the RHS value of 5

0.6


0.6


0.6


0.6


0.6


But for our user defined functions we can only make use of the functional way of calling:

In [42]:
divide:{[numerator;denominator] numerator%denominator} 

divide[3;5]    //functionally calling
divide[3][5]
divide[3] 5 
divide[;5] 3
divide[;5][3]
3 divide 5     //infix calling throws an error

0.6


0.6


0.6


0.6


0.6


[0;31mtype[0m: [0;31mtype[0m

<img src="../qbies.png" style="width: 50px;padding-right:5px;padding-top:10px;padding-left:5px;" align="left"/>

<p style='color:#273a6e'><i> Remember the number of input parameter(s) in the application of a function must match the definition. An application with too many arguments generates a `rank error. An application with too little arguments turns the function into a projection</i></p>

##### Exercise

Create a function with three parameters `x`,`y` and `z`. The function logic should return `x` plus `y` multiplied by `z`

In [ ]:
func1:{[x;y;z] (x+y)*z}

In [45]:
// Enter your Q code here 
fun:{[x;y;z] z*x+y}
fun[1;5;4]

24


Create a function that produces `n` random integers between 0-100 where `n` is a parameter

In [ ]:
func2:{[n] n?101i}  //specifing 101 as we want to include 100 

In [46]:
// Enter your Q code here 
rand2:{[n] n?101i}
rand2[3]

93 54 38i


# Explicit and Implicit parameters

## Explicit Parameters

To explicitly declare the parameters to a function we define them within the square brackets immediately after the
opening curly bracket, and if there is more than one parameter to be declared, we separate these parameters
with semicolons `;`. For example:

In [47]:
sumStats:{[l] //single input function with 4 expressions
    countL:count l; //calculating number of elements in list 
    minL:min l; //calculating min of list 
    maxL:max l; //calculating max of list
    avgL:avg l; //calculating average of list
    :(countL;minL;maxL;avgL) //using force return 
 }

In [48]:
show L:10?100 //creating random list
show s1:sumStats[L] //execute the function and assign result to global variable s1

97 88 58 68 45 2 39 64 49 82
10
2
97
59.2


##### Exercise
Create a function `rectangleArea1` that returns the area of a rectangle using two explicit parameters, length and width. 

Using the function, calculate the area of a rectangle with length 5.93 and width 3.87.  

In [ ]:
rectangleArea1:{[length;width] length * width}
rectangleArea1[5.93;3.87]

In [49]:
// write your function here
rectangleArea1:{[l;w] l*w}
rectangleArea1[5.93;3.87]

22.9491


## Implicit Parameters
If there are no parameters explicitly declared at the start of a function definition (no square
brackets), then up to three inputs can be implicitly available if defined within the function body. These are by
name `x`,`y` and `z`.

In [50]:
show f4:{x-1} //Function takes a single input and implicitly returns a result
f4[1 2]

{x-1}


0 1


In [51]:
show f5:{r:x-y;
    :r} //Function takes 2 inputs and explicitly returns a result
f5[3;2]

{r:x-y;
    :r}


1


In [52]:
show f6:{:x-y*z} //Function takes 3 inputs and explicitly returns a result
f6[10;20;f5[3;2]]

{:x-y*z}


-10


This also only works for these three values, trying to do this with `a`,`b` and `c` would be unsuccessful.

##### Exercise 
Create a function `rectangleArea2` to find the area of a rectangle with length 7.93 and width 1.87 using implicit parameters

In [ ]:
rectangleArea2:{x*y}  //using implicit parameters
rectangleArea2[7.93;1.87]

In [60]:
/ write your function here
rectangleArea2:{x*y} //only can use x y z
rectangleArea2:{l*w} //gives error 
rectangleArea2[7.93;1.87]

[0;31mrank[0m: [0;31mrank[0m

##  Comparison
Let's suppose we want to define a function called `getCircleArea`, which takes the radius as its sole argument and returns the area of such a circle. For the purposes of this exercise we will assume π=3.14159

In [ ]:
getCircleArea:{[radius] 3.14159*radius*radius};   // using explicit argument here, 
                                                  // which must be defined at the start of the lambda in square brackets
getCircleArea 4

If we chose not to explicitly declare the argument and instead used the implicit argument x:

In [ ]:
getCircleArea:{3.14159*x*x};   // use x instead of explicit argument, no square brackets required
getCircleArea 4                // function behaviour is unchanged

##### Exercise 
Create a monadic function to find  all the odd numbers in a list and multiple them by 10.

In [ ]:
oddFunc:{[i]
    oddNum:i where 1=i mod 2; //finding all the odd numbers
    @[i;oddNum;*;10]} //multiple by 10
oddFunc[1 2 3 4]

In [58]:
// your answer here
oddf:{[l] 10*l where 1=l mod 2}
oddf[1 2 3 4]


10 30


Create a dyadic function `vol` that takes a radius and a length to calculate the volume of a cylinder ($V=l*\pi r^2$ - Use 3.14 for $\pi$)

In [ ]:
vol:{y*3.14*x*x}
vol[2;3]

In [ ]:
// your answer here
vol:{[r,l] l*3.14*r2}

# Function Scope

Within programming languages that utilize functions, there is the concept of [function scope](https://en.wikipedia.org/wiki/Scope_(computer_science)#Function_scope) which refers to the "view" of the variables that are accessible to a function during it's execution. 

## Local Scope

The *local scope* of a function refers to the variables that are available and accessible within the execution of a function. Variables which only exist for the duration of the function execution and fail to be accessible after function execution are deemed to be *local* to the function execution, and therefore *local variables*.

The most simple example of variables that are local only to the scope of a function are the parameters of function: 

In [ ]:
f:{[param1] -1 "Parameter passed: ",string param1; param1*2}   
f[2]
param1     //not defined outside of our function in what is called the "global" scope

Another example of local variables are the intermediate variables that we might define within our function logic: 

In [ ]:
f:{local:42; -1"Value of local: ",string local; local+x} // "local" is a local variable
f[10]
local

 A local variable is not visible within the body of a local function defined within the same function and same scope.
We can best clarify what exactly is meant by this with an example: 

In [ ]:
f:{localToF:21;      //create variable within the scope of function f
    g:{localToF*2}; //create new function within the scope of our function 
    g[]             //call g within the function f
  }            
f[]     //g can't access  the localvariable localToF

If we wanted `g` to have access to that variable we would need to pass it as a parameter as follows: 

In [ ]:
f:{localToF:21;                               //create variable within the scope of function f
    g:{[passedFromFtoG] passedFromFtoG*2};    //create new function within the scope of our function 
    g[localToF]                               //call g within the function f passing our local variable as an input
  }            
f[]

##### Exercise 
Create a function `circumferenceOfCircle` that defines a local variable `pi:3.14` and finds the [circumference](https://www.mathplanet.com/education/pre-algebra/more-about-equation-and-inequalities/calculating-the-circumference-of-a-circle) of a circle. 

Calculate the circumference of a circle with radius 6. Check to see if pi is defined in the process.

In [ ]:
circumferenceOfCircle:{[r]
  pi:3.14; //defining local variable
  2*pi*r  //calculating circumference
 }

circumferenceOfCircle[6]
pi //inspecting pi - not defined as it's a local variable

In [63]:
//write your function here
circumferenceOfCircle:{[r] pi:3.14;
                            2*pi*r}
circumferenceOfCircle[6]

37.68


## Global Scope

A variable assigned outside of a function is called a *global variable*

In [64]:
f:{b:2; b*x}  //defining locally
f[6]

12


When a function calls for a variable, it first "looks" locally, and then if the variable can't be found in local scope, the function "looks" globally. If the variable can't be found in either scope then the function will error.  

In [64]:
b:4
f:{b*x}          //calling b but we haven't defined it locally 
f[1]
delete b from `. //removing b from the global scope 
f[1]             //b now undefined 

4


`.


[0;31mb[0m: [0;31mb[0m

If we wanted to define global variables from within our function, we can do so using `set` (call-by-name) or double colon assignment `::`, provided there is no local variable with the same name.

In [65]:
delete b, c from `.  //clearing b and c from global scope 
{`b set 47;c::74;}[] // We can see both b & c are defined outside the function after execution
b
c

`.


47


74


If you try to use `::` to set a global variable of the same name as a local variable, then you only succeed in updating the local scope. In the next example we see local-global name collision using `::` which doesn't throw errors

In [66]:
/setting b:6 globally
b:6
/assigning b:7 globally
f:{b::7; x*b}

f[10] //x*b = 10*7 = 70

/b:7 assigned globally
b     // using :: (no local-global name collision): assign successfully as global

70


7


In [68]:
/setting b:6 globally
b:6                  
/setting b:42 locally, attempt to set b::x globally but unable to due to collision, only sets it locally
f:{b:42; b::x; b}
/b:42 locally, b:98 locally(unable to set globally), output of b is then 98
f[98]
/b remains as 6
b      // using :: (local-global name collision): assigns the local, not the global 

98


6


What would happen in the above example if we used `set` to define the global variable from inside the function call instead of using `::` ?

In [67]:
/resetting b:6 globally
b:6                  
/setting b:42 locally, setting b set x globally and doesn't overwrite the local variable
f:{b:42; `b set x; b}

/b:42 locally, b:98 set globally, output of function is local variable b is 42 
f[98]
/b globally changes to 98
b      // using set assigns the global

42


98


##### Exercise 

Create a function that creates a global variable `LAST_CALLED` which stores the time that the function was last called at. 

This function should take two inputs, and return the result of multiplying them together.

In [ ]:
globalFunc:{LAST_CALLED::.z.t;x*y} //setting the global value to current time

globalFunc[2;3] //calling the new function
LAST_CALLED //inspecting last_called

In [71]:
//your answer here
fun1:{[x;y]LAST_CALLED::.z.t;x*y}
fun1[2;5]
LAST_CALLED

10


12:40:53.307


# Projections 

A projection is created by passing a function less than it's total required arguments. It is used when we want to keep one or more of the arguments constant.

## Creating Projections

Let's take the example of the addition operator, `+`, which takes 2 arguments. If we want to create a projection from the `+` operator, we must pass in 1 argument as opposed to 2.

In [73]:
addTwo:2+ //creating a projection - binding the first input to be 2 
addTwo
addTwo 1 2 4 3.2

+[2]


3 4 6 5.2


##### Using a primitive operator

Looking at creating a projection from a primative operator, let's take the following as an example:

In [74]:
9+5                    // normal operation of + using infix notation
+[9;5]                 // can also be called with square bracket notation, passing in 2 arguments

14


14


If we wanted to create a new function that would add 9 to each argument passed to the function, we could create a projection like follows:

In [75]:
add9a:9+                  //using infix notation
add9b:+[9]               //using the square bracket notation
add9a
add9b

+[9]


+[9]


##### Creating a projection using a lambda 

 What happens if we have a lambda that we would like create a projection from ?

In [78]:
add:{x+y}
add4:add 4             //calling a dyadic function with one argument creates a projection
add4
add4 10

{x+y}[4]


14


##### Projecting other arguments
Until now, we have only projected the first argument of a function - however we can project any argument in q! 

We can show how this can be done using the primitive operator `%` to create a projection that will half each argument passed:

In [78]:
half:%2 // infix notation - in this case, y is the argument that we are projecting so this errors

[0;31mparse error[0m: [0;31m%[0m

We have to use square bracket notation if we want to project arguments that aren't the first argument. We do this in a similar way to index elision. 

In [79]:
half:%[;2]  // to create a projection where the second argument is held constant, simply omit the first argument
half 3

1.5


##### Exercise  
Create a projection called `modSeven` that calculates the input modulo 7. 

Check the projection with an input 15

In [ ]:
modSeven:mod[;7]
modSeven 15 //checking it works

In [80]:
// write your projection here
modSeven:mod[;7]
modSeven 15

1


## Applying projections
Let's take a look at the projections that we created in the previous section `add9` and `add4`.

In [81]:
add9a
add4

+[9]


{x+y}[4]


In [82]:
add9a 10    // we can now re-use this projection and pass in a 2nd argument
add4 10 
add9a 10 20 30 //will work for lists 
add4 10 20 30

19


14


19 29 39


14 24 34


##### Exercise 
Using the projection `modSeven`, apply it to the series 0-10.

In [ ]:
modSeven til 11 

In [83]:
// write your answer here
modSeven til 11

0 1 2 3 4 5 6 0 1 2 3
